### Imports 

In [5]:
import boto3
from datetime import datetime as dt
import os
import pandas as pd
import time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)

# Set ClientId

In [6]:
client_id = 'BCMA'
received_date = '2019-01-01'

client_id = client_id.lower()
received_date = 'RD-'+pd.to_datetime(received_date).strftime("%Y-%m-%d")
database = client_id+'_'+received_date

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Starting with S3

In [82]:
session = boto3.Session(region_name='us-east-1')
s3_resource = session.resource('s3')
s3_client = s3_resource.meta.client

###  1. Creating Raw Folder Structure

This folder structure is ...

In [83]:
def create_s3_folder_structure(received_date, folder_list):
    for folder in folder_list:
        received_date_key = received_date
        complete_structure = os.path.join(client_id, received_date_key, folder, '')
        s3_client.put_object(Bucket='prospect-raw-files', Key=complete_structure)

In [99]:
folders = ['ClaimtData', 'MembershipData', 'ControlData']
s3_path = os.path.join('s3://prospect-raw-files/', client_id, received_date)
create_s3_folder_structure(received_date, folders)

# UPLOAD THE FILES TO S3

---
# Continue with Glue Crawler

In [85]:
glue_client = boto3.client('glue', region_name='us-east-1')

### 1. Creating Database

In [88]:
glue_client.create_database(
    DatabaseInput={
        'Name': database
    }
)

{'ResponseMetadata': {'RequestId': '6bb501c6-fb84-11e9-95fc-35be55017697',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 31 Oct 2019 02:16:17 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6bb501c6-fb84-11e9-95fc-35be55017697'},
  'RetryAttempts': 0}}

### 2. Creating Crawler

In [92]:
crawler_name = client_id.lower()+'-raw-crawler_'+received_date
glue_client.create_crawler(
    Name=crawler_name,
    Role='UnderwritingServiceRole',
    DatabaseName=database.lower(),
    Description='Automated Single Usage - Delete after use',
    Targets={
        'S3Targets': [
            {'Path': s3_path}
        ]
    }
)

{'ResponseMetadata': {'RequestId': '2fdf3468-fb85-11e9-b20a-8b7506446ea4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 31 Oct 2019 02:21:46 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2fdf3468-fb85-11e9-b20a-8b7506446ea4'},
  'RetryAttempts': 0}}

### 3. Running Crawler

In [93]:
glue_client.start_crawler(Name=crawler_name)
time.sleep(60)
print("It should be stopping now!")

{'ResponseMetadata': {'RequestId': '36a0076b-fb85-11e9-b498-a115b0e9e017',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 31 Oct 2019 02:22:00 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '36a0076b-fb85-11e9-b498-a115b0e9e017'},
  'RetryAttempts': 0}}

### 4. Delete Crawler

In [104]:
glue_client.delete_crawler(Name=crawler_name)

{'ResponseMetadata': {'RequestId': '5d7ddd7d-fb86-11e9-b2c9-69912831a3e0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 31 Oct 2019 02:30:12 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5d7ddd7d-fb86-11e9-b2c9-69912831a3e0'},
  'RetryAttempts': 0}}

---
# Continue with Glue Jobs in Spark Clusters

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
22,application_1572457794802_0023,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 1. Getting Catalogue Content

In [15]:
claim_data = glueContext.create_dynamic_frame.from_catalog(database=database, 
                                                           table_name='clientdata')
member_data = glueContext.create_dynamic_frame.from_catalog(database=database, 
                                                            table_name='membershipdata')
control_data = glueContext.create_dynamic_frame.from_catalog(database=database, 
                                                             table_name='controlfile')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2. Exploring Schema

In [16]:
claim_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- clm_no: long
|-- clm_no_ln: long
|-- serv_dt: string
|-- resolved_dt: string
|-- allow_amt: double
|-- resolved_amt: double

In [17]:
member_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- mem_month: string
|-- mem_qty: long

In [18]:
control_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- columnname: string
|-- sum: double
|-- count: double

### Row Count

In [29]:
print(f'Claim Rows: {claim_data.count():2,.0f}', 
      f'Member Rows: {member_data.count():2,.0f}', sep='\n')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Claim Rows: 3,099
Member Rows: 34

In [22]:
control_data.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------+------+
|  columnname|              sum| count|
+------------+-----------------+------+
|      CLM_NO|  1.0330051527E10|1033.0|
|   CLM_NO_LN|           6180.0|1033.0|
|     SERV_DT|             null|1033.0|
| RESOLVED_DT|             null|1033.0|
|   ALLOW_AMT|514550.9463331801|1033.0|
|RESOLVED_AMT|72037.13248664522|1033.0|
+------------+-----------------+------+

## Membership

### Schema

### Row Count

## Control

### Schema

### Row Count